<a href="https://colab.research.google.com/github/restrepo/OpenAlex/blob/main/json_total.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAlex works for Colombia
## Concepts associated to "Physics" in UdeA

In [1]:
import json
import pandas as pd
from google.colab import files
#From https://gist.github.com/korakot/072f1ea70d3c3267ff2c64b61d78c89c
import json
from IPython.display import HTML

def JSON(jstr):
  if type(jstr) != str:
    jstr = json.dumps(jstr)
  return HTML("""
<script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
<script>
renderjson.set_show_to_level(1)
document.body.appendChild(renderjson(%s))
new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
</script>
""" % jstr)

### Load the json file of works for Colombia

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
with open('/content/drive/MyDrive/json/openalexco.json','r') as miarchivo:
    objeto=miarchivo.read()
objeto=json.loads(objeto)

In [4]:
len(objeto)

343057

Filter works with ror:

https://ror.org/03bp5hc83

In [5]:
objeto=[ c for c in objeto if any(
    [ any([i.get('ror')=="https://ror.org/03bp5hc83" for i in a.get('institutions')]) 
    for a in c.get('authorships')])
]
len(objeto)

34291

Define the filter which returns `True or False`

In [ ]:
def filter_ror(authorships,ror="https://ror.org/03bp5hc83"):
  return any([any([i.get('ror')=="https://ror.org/03bp5hc83" for i in a.get('institutions')])  for a in authorships])

In [ ]:
len([ c for c in objeto if filter_ror(c.get('authorships'))])

34291

Filter with Pandas

In [6]:
objeto=pd.DataFrame(objeto)

In [ ]:
objeto[objeto['authorships'].apply(filter_ror)].shape

(34291, 25)

### Explore concept levels from Excel file

In [ ]:
cp=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vT9JrvUwanM-z-hCZ1SSaxRx2wfzLV3RVvm5BJaf8TABBmpIwKt7iYt1sVLbImVevh9YNaZK9Y7sfI9/pub?output=xlsx')

In [ ]:
cp[cp['display_name'].str.lower().str.contains('complex')][:3]#['display_name']#.to_list()

,openalex_id,display_name,normalized_name,level,wikidata_id,parent_display_names,parent_ids
377,https://openalex.org/c34947359,Complex network,complex network,2.0,https://www.wikidata.org/wiki/Q665189,"Combinatorics, World Wide Web","https://openalex.org/C114614502, https://opena..."
1241,https://openalex.org/c2778445095,Sample complexity,sample complexity,2.0,https://www.wikidata.org/wiki/Q18354077,NaN,NaN
2527,https://openalex.org/c47822265,Complex system,complex systems,2.0,https://www.wikidata.org/wiki/Q854457,"Artificial intelligence, Social science","https://openalex.org/C154945302, https://opena..."


#### Level 1: Physics concepts
The 'parent_display_names' column has some `NaN`, which is a `float`, and therefore must be converted to `str`, before the string operations to avoid errors. `Physics'` may appears along with other concepts in the `'parent_display_names'` and therefore the `str.contains` method of Pandas is the best option here:

In [ ]:
ph1=cp[ (cp['level']==1) & (cp['parent_display_names'].astype(str).str.contains('Physics')) 
      ]['display_name'].to_list()

In [ ]:
len(ph1)

27

#### Level 2: Physics concepts
the `str.split` method of Pandas, conver the comma-space sepparated list into a Python list. Remember that the `set` object of Python has the `intersection` method directly

In [ ]:
ph2=cp[(cp['level']==2) & (cp['parent_display_names'].astype(str).str.split(', ').apply(
    lambda l: set(l).intersection(set(ph1))).apply(len)>0)]['display_name'].to_list()
len(ph2)

2979

### UdeA Physics articles

In [ ]:
concept='Physics'
level=0
possible_physics=objeto[objeto['concepts'].apply(lambda l: {concept}.intersection( 
    [d.get('display_name') for d in l if d.get('level')==level])).apply(len)>0]
possible_physics.shape


(1886, 25)

Level 1: UdeA Physics concepts

In [ ]:
ll=possible_physics['concepts'].apply(lambda l: set([d.get('display_name') for d in l if d.get('level')==1]).intersection(ph1)).apply(list)
ph1_udea=set([item for subitem in ll for item in subitem])

Missing Level 1 concepts at UdeA

In [ ]:
set(ph1).difference(ph1_udea)

{'Geophysics', 'Medical physics'}

Level 2: UdeA Physics concepts

In [ ]:
ll=possible_physics['concepts'].apply(lambda l: set([d.get('display_name') for d in l if d.get('level')==2]).intersection(ph2)).apply(list)
ph2_udea=set([item for subitem in ll for item in subitem])

In [ ]:
len(ph2_udea)

507

### UdeA Physics concepts tree

In [ ]:
d={'Physics': dict( [(c,[]) for c in ph1_udea])}
for c in ph2_udea:
    lph1=set(ph1_udea).intersection( cp[cp['display_name']==c]['parent_display_names'].str.split(', ').iloc[0])
    for cc in lph1:
        if cc in d['Physics'] and c not in d['Physics'][cc]:
            d['Physics'][cc].append(c)
del cp


In [ ]:
JSON(d)

### Generic filter

In [ ]:
if isinstance(objeto,list):
    objeto=pd.DataFrame(objeto)
concept='Complex systems'
level=2
def filter_concept(df,concept,level):
    return df[df['concepts'].apply(lambda l: {concept}.intersection( 
    [d.get('display_name') for d in l if d.get('level')==level])).apply(len)>0]

In [ ]:
filter_concept(objeto,concept,level)

,id,doi,title,display_name,publication_year,publication_date,ids,host_venue,type,open_access,...,concepts,mesh,alternate_host_venues,referenced_works,related_works,cited_by_api_url,counts_by_year,updated_date,created_date,abstract_inverted_index


In [ ]:

f=open('openalexua.json','w')
json.dump(objeto,f)
f.close()

In [ ]:

x=set()
type(x)

In [ ]:
f=open('openalexua.json','r')
df=json.load(f)
f.close()

Filter author

In [9]:
objeto['authorships'].iloc[1]

[{'author_position': 'first',
  'author': {'id': 'https://openalex.org/A4286489923',
   'display_name': 'Edwin A. Alarcón',
   'orcid': None},
  'institutions': [{'id': 'https://openalex.org/I35961687',
    'display_name': 'University of Antioquia',
    'ror': 'https://ror.org/03bp5hc83',
    'country_code': 'CO',
    'type': 'education'}],
  'raw_affiliation_string': 'Universidad de Antioquia, Chemical Engineering Department, Environmental Catalysis Research Group, Calle 70 No. 52-21, Medellín, Colombia'},
 {'author_position': 'last',
  'author': {'id': 'https://openalex.org/A4286489924',
   'display_name': 'Aída Luz Villa',
   'orcid': None},
  'institutions': [],
  'raw_affiliation_string': None}]

In [39]:
au_id='https://openalex.org/A2066982801'
df=objeto[objeto['authorships'].apply(lambda l: any([ d.get('author').get('id')==au_id for d in l]))]
df.shape

(24, 25)

In [36]:
l=df['concepts'].apply(lambda l: [d.get('display_name') for d in l if d.get('level')==0 or d.get('level')==1]).to_list()
set([item for subitem in l for item in subitem ])

{'Algorithm',
 'Artificial intelligence',
 'Biochemistry',
 'Bioinformatics',
 'Biological system',
 'Biology',
 'Biophysics',
 'Chemical physics',
 'Chemistry',
 'Combinatorial chemistry',
 'Computational biology',
 'Computational chemistry',
 'Computer science',
 'Condensed matter physics',
 'Crystallography',
 'Data mining',
 'Data science',
 'Genetics',
 'Machine learning',
 'Materials science',
 'Molecular physics',
 'Nanotechnology',
 'Physics',
 'Programming language',
 'Statistical physics',
 'Stereochemistry',
 'Theoretical computer science'}

In [37]:
l=df['concepts'].apply(lambda l: [d.get('display_name') for d in l if d.get('level')==1]).to_list()
set([item for subitem in l for item in subitem ])

{'Algorithm',
 'Artificial intelligence',
 'Biochemistry',
 'Bioinformatics',
 'Biological system',
 'Biophysics',
 'Chemical physics',
 'Combinatorial chemistry',
 'Computational biology',
 'Computational chemistry',
 'Condensed matter physics',
 'Crystallography',
 'Data mining',
 'Data science',
 'Genetics',
 'Machine learning',
 'Molecular physics',
 'Nanotechnology',
 'Programming language',
 'Statistical physics',
 'Stereochemistry',
 'Theoretical computer science'}

In [38]:
['Biophysics',
 'Chemical physics',
 'Molecular physics',
 'Statistical physics',
]

['Biophysics', 'Chemical physics', 'Molecular physics', 'Statistical physics']

In [75]:
df['concepts'].apply(lambda l: set([d.get('display_name') for d in l if d.get('level')==1 or d.get('display_name').find('Physics')>-1]).intersection(
    ['Physics','Biophysics',
 'Chemical physics',
 'Molecular physics',
 'Statistical physics',
 'Computational biology'
]
   )).reset_index(drop=True)

0     {Molecular physics, Chemical physics, Statisti...
1                                             {Physics}
2                                                    {}
3                               {Computational biology}
4                               {Computational biology}
5                               {Computational biology}
6                               {Computational biology}
7                               {Computational biology}
8                               {Computational biology}
9                               {Computational biology}
10                                                   {}
11                              {Computational biology}
12                              {Computational biology}
13                                                   {}
14                              {Computational biology}
15                                                   {}
16                                                   {}
17                              {Computational b

In [87]:
objeto[objeto['concepts'].apply(lambda l: set([d.get('display_name') for d in l if d.get('level')==1 #or d.get('display_name').find('Physics')>-1
                                               ]
                                              ).intersection(
    ['Biophysics',
 'Molecular physics',
]
   )).apply(len)>0]

,id,doi,title,display_name,publication_year,publication_date,ids,host_venue,type,open_access,...,concepts,mesh,alternate_host_venues,referenced_works,related_works,cited_by_api_url,counts_by_year,updated_date,created_date,abstract_inverted_index
48,https://openalex.org/W4288712741,https://doi.org/10.4028/p-gh5bxa,Uncorrelated Excitonic Properties in Multilaye...,Uncorrelated Excitonic Properties in Multilaye...,2022,2022-07-29,{'openalex': 'https://openalex.org/W4288712741...,"{'id': 'https://openalex.org/V187696258', 'iss...",journal-article,"{'is_oa': False, 'oa_status': None, 'oa_url': ...",...,"[{'id': 'https://openalex.org/C124657808', 'wi...",[],[],"[https://openalex.org/W30010175, https://opena...","[https://openalex.org/W142829, https://openale...",https://api.openalex.org/works?filter=cites:W4...,[],2022-07-30T12:25:38.793882,2022-07-30,NaN
190,https://openalex.org/W4281974086,https://doi.org/10.1016/j.bioadv.2022.212967,Viscoelastic properties of plasma-agarose hydr...,Viscoelastic properties of plasma-agarose hydr...,2022,2022-06-01,{'openalex': 'https://openalex.org/W4281974086...,"{'id': 'https://openalex.org/V4210239156', 'is...",journal-article,"{'is_oa': True, 'oa_status': 'hybrid', 'oa_url...",...,"[{'id': 'https://openalex.org/C108586683', 'wi...","[{'descriptor_ui': 'D020100', 'descriptor_name...","[{'id': 'https://openalex.org/V4210239156', 'd...","[https://openalex.org/W2003729795, https://ope...","[https://openalex.org/W1545117257, https://ope...",https://api.openalex.org/works?filter=cites:W4...,"[{'year': 2022, 'cited_by_count': 1}]",2022-08-12T14:02:38.872994,2022-06-13,NaN
227,https://openalex.org/W4281258132,https://doi.org/10.1007/s11082-022-03758-w,Shallow-donor impurity effects on the far infr...,Shallow-donor impurity effects on the far infr...,2022,2022-05-22,{'openalex': 'https://openalex.org/W4281258132...,"{'id': 'https://openalex.org/V133811445', 'iss...",journal-article,"{'is_oa': False, 'oa_status': 'closed', 'oa_ur...",...,"[{'id': 'https://openalex.org/C124657808', 'wi...",[],[],"[https://openalex.org/W945548251, https://open...","[https://openalex.org/W3828417, https://openal...",https://api.openalex.org/works?filter=cites:W4...,[],2022-07-08T14:55:29.277828,2022-05-23,NaN
231,https://openalex.org/W4280619639,https://doi.org/10.3390/membranes12050534,Infrared Spectroscopic Study of Multi-Componen...,Infrared Spectroscopic Study of Multi-Componen...,2022,2022-05-20,{'openalex': 'https://openalex.org/W4280619639...,"{'id': 'https://openalex.org/V4210225467', 'is...",journal-article,"{'is_oa': True, 'oa_status': 'gold', 'oa_url':...",...,"[{'id': 'https://openalex.org/C41625074', 'wik...",[],"[{'id': 'https://openalex.org/V4210225467', 'd...","[https://openalex.org/W28854146, https://opena...","[https://openalex.org/W4828911, https://openal...",https://api.openalex.org/works?filter=cites:W4...,"[{'year': 2022, 'cited_by_count': 1}]",2022-07-20T06:16:02.421620,2022-05-22,NaN
238,https://openalex.org/W4280611236,https://doi.org/10.1021/acsomega.1c06515,Cytotoxicity Assessment of Surface-Modified Ma...,Cytotoxicity Assessment of Surface-Modified Ma...,2022,2022-05-19,{'openalex': 'https://openalex.org/W4280611236...,"{'id': 'https://openalex.org/V4210239500', 'is...",journal-article,"{'is_oa': True, 'oa_status': 'gold', 'oa_url':...",...,"[{'id': 'https://openalex.org/C2777230088', 'w...",[],"[{'id': 'https://openalex.org/V4210239500', 'd...","[https://openalex.org/W1579782115, https://ope...","[https://openalex.org/W11361684, https://opena...",https://api.openalex.org/works?filter=cites:W4...,[],2022-07-20T06:18:48.038986,2022-05-22,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32069,https://openalex.org/W2024928190,https://doi.org/10.1088/0268-1242/14/6/302,Theoretical study of the density of shallow-ac...,Theoretical study of the density of shallow-ac...,1999,1999-06-01,{'openalex': 'https://openalex.org/W2024928190...,"{'id': 'https://openalex

In [74]:
Çdf.concepts.iloc[20]

[{'id': 'https://openalex.org/C154945302',
  'wikidata': 'https://www.wikidata.org/wiki/Q11660',
  'display_name': 'Artificial intelligence',
  'level': 1,
  'score': '0.41079968'},
 {'id': 'https://openalex.org/C75806775',
  'wikidata': 'https://www.wikidata.org/wiki/Q5358194',
  'display_name': 'Electron tomography',
  'level': 4,
  'score': '0.40721047'},
 {'id': 'https://openalex.org/C20702342',
  'wikidata': 'https://www.wikidata.org/wiki/Q5190506',
  'display_name': 'Cryo-electron microscopy',
  'level': 2,
  'score': '0.40552735'},
 {'id': 'https://openalex.org/C193016168',
  'wikidata': 'https://www.wikidata.org/wiki/Q874835',
  'display_name': 'Scanning transmission electron microscopy',
  'level': 3,
  'score': '0.3849336'},
 {'id': 'https://openalex.org/C186060115',
  'wikidata': 'https://www.wikidata.org/wiki/Q30336093',
  'display_name': 'Biological system',
  'level': 1,
  'score': '0.3525777'},
 {'id': 'https://openalex.org/C153180895',
  'wikidata': 'https://www.wikidat